## 環境設置

In [4]:
# Mount to Google Drive
# from google.colab import drive
# # drive.mount('/content/drive', force_remount=True)

In [5]:
!pip install nnunetv2 -q


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import os

os.environ['nnUNet_raw'] = './nnU_Base/nnUNet_raw_data'
os.environ['nnUNet_preprocessed'] = './nnU_Base/nnUNet_preprocessed'
os.environ['nnUNet_results'] = './nnU_Base/nnUNet_trained_models'

## 用程式碼上傳raw

In [7]:
import os
import shutil
import time
import gc
from tqdm import tqdm

def copy_single_file(src_file, dest_file):
    """Copy a single file with error handling"""
    try:
        # Create parent directories if they don't exist
        os.makedirs(os.path.dirname(dest_file), exist_ok=True)

        # Copy the file
        shutil.copy2(src_file, dest_file)
        return True
    except Exception as e:
        print(f"Error copying {src_file} to {dest_file}: {str(e)}")
        return False

def get_all_files(source_dir):
    """Get a list of all files in the directory and its subdirectories"""
    all_files = []
    for root, _, files in os.walk(source_dir):
        for file in files:
            src_file = os.path.join(root, file)
            all_files.append(src_file)
    return all_files

def robust_copy_directory(source_dir, dest_dir, start_idx=0, batch_size=5, sleep_time=2):
    """
    Copy files from source_dir to dest_dir with memory management

    Args:
        source_dir: Source directory
        dest_dir: Destination directory
        start_idx: Index to start copying from (useful for resuming)
        batch_size: Number of files to copy before pausing
        sleep_time: Time to sleep between batches in seconds
    """
    # Create destination directory if it doesn't exist
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    # Get list of all files
    all_files = get_all_files(source_dir)
    total_files = len(all_files)

    if start_idx >= total_files:
        print(f"Start index {start_idx} is greater than total files {total_files}")
        return

    print(f"Found {total_files} files to copy, starting from index {start_idx}")

    # Process files in small batches
    for i in range(start_idx, total_files, batch_size):
        batch_end = min(i + batch_size, total_files)
        print(f"\nProcessing batch {i//batch_size + 1}: files {i+1}-{batch_end} of {total_files}")

        # Process each file in the current batch
        for j in range(i, batch_end):
            src_file = all_files[j]
            # Calculate relative path to maintain directory structure
            rel_path = os.path.relpath(src_file, source_dir)
            dest_file = os.path.join(dest_dir, rel_path)

            print(f"Copying file {j+1}/{total_files}: {rel_path}")
            success = copy_single_file(src_file, dest_file)

            # Force garbage collection after each file
            gc.collect()

        # After each batch, print progress and pause
        print(f"Completed {batch_end}/{total_files} files ({(batch_end/total_files)*100:.1f}%)")

        if batch_end < total_files:
            print(f"Pausing for {sleep_time} seconds before next batch...")
            time.sleep(sleep_time)

            # More aggressive garbage collection every 50 files
            if (batch_end % 50) < batch_size:
                print("Performing extensive garbage collection...")
                for _ in range(3):
                    gc.collect()
                time.sleep(5)

            # Print resume information
            print(f"""
If this process freezes, you can resume by running:
robust_copy_directory("{source_dir}", "{dest_dir}", start_idx={batch_end}, batch_size={batch_size})
""")

    print(f"\nCopy complete! Copied {total_files} files from {source_dir} to {dest_dir}")

def verify_copy(source_dir, dest_dir):
    """Verify that all files were copied correctly"""
    source_files = get_all_files(source_dir)
    dest_files = get_all_files(dest_dir)

    # Convert to relative paths for comparison
    source_rel_paths = set(os.path.relpath(f, source_dir) for f in source_files)
    dest_rel_paths = set(os.path.relpath(f, dest_dir) for f in dest_files)

    missing_files = source_rel_paths - dest_rel_paths

    print(f"Source files: {len(source_rel_paths)}")
    print(f"Destination files: {len(dest_rel_paths)}")

    if missing_files:
        print(f"WARNING: {len(missing_files)} files were not copied!")
        if len(missing_files) < 10:
            print("Missing files:")
            for f in missing_files:
                print(f"  - {f}")
        else:
            print("First 10 missing files:")
            for f in list(missing_files)[:10]:
                print(f"  - {f}")
    else:
        print("All files were copied successfully!")

In [8]:
source_directory = './nnU_Base/nnUNet_raw_data/Dataset001_MYTASK'
destination_directory = './nnU_Base/nnUNet_raw_data/Dataset001_MYTASK'

# robust_copy_directory(source_directory, destination_directory)

# # After copying completes, verify that everything was copied correctly
# verify_copy(source_directory, destination_directory)

In [9]:
import os
import json

def generate_dataset_json(output_folder: str,
                          channel_names: dict,
                          labels: dict,
                          num_training_cases: int,
                          file_ending: str,
                          dataset_name: str = None,
                          overwrite_image_reader_writer: str = None,
                          ):
  """
  製作如下的json檔
  {
      "name": "nnunet-how-to",
      "channel_names": {
          "0": "CT"
      },
      "labels": {
          "background": 0,
          "AA": 1
      },
      "numTraining": 10,
      "file_ending": "nii.gz",
      "overwrite_image_reader_writer": "SimpleITKIO"
  }
  """
  # channel names need strings as keys
  keys = list(channel_names.keys())
  for k in keys:
      if not isinstance(k, str):
          channel_names[str(k)] = channel_names[k]
          del channel_names[k]

  # labels need ints as values
  for l in labels.keys():
      value = labels[l]
      if isinstance(value, (tuple, list)):
          value = tuple([int(i) for i in value])
          labels[l] = value
      else:
          labels[l] = int(labels[l])

  dataset_json = {
      'name': '',
      'channel_names': channel_names,
      'labels': labels,
      'numTraining': num_training_cases,
      'file_ending': file_ending,
      'overwrite_image_reader_writer': ''
  }

  if dataset_name is not None:
      dataset_json['name'] = dataset_name
  if overwrite_image_reader_writer is not None:
      dataset_json['overwrite_image_reader_writer'] = overwrite_image_reader_writer

  save_path = os.path.join(output_folder, 'dataset.json')
  with open(save_path, 'w') as f:
      json.dump(dataset_json, f, indent=4)

  print(f'dataset.json saved to {save_path}')

In [10]:

json_dir_path = './nnU_Base/nnUNet_raw_data/Dataset001_MYTASK'
generate_dataset_json(json_dir_path,
                      {0: 'CT'},                              # channel_names
                      {"background": 0, "AA": 1},    # labels
                      300,                                    # num_training_cases
                      '.nii.gz',                              # file_ending
                      overwrite_image_reader_writer='SimpleITKIO'
                      )

dataset.json saved to ./nnU_Base/nnUNet_raw_data/Dataset001_MYTASK\dataset.json


## plan and preprocess

nnUNetv2_plan_and_preprocess = nnUNetv2_extract_fingerprint + nnUNetv2_plan_experiment + nnUNetv2_preprocess

In [11]:
!nnUNetv2_plan_and_preprocess -h

usage: nnUNetv2_plan_and_preprocess [-h] [-d D [D ...]] [-fpe FPE]
                                    [-npfp NPFP] [--verify_dataset_integrity]
                                    [--no_pp] [--clean] [-pl PL]
                                    [-gpu_memory_target GPU_MEMORY_TARGET]
                                    [-preprocessor_name PREPROCESSOR_NAME]
                                    [-overwrite_target_spacing OVERWRITE_TARGET_SPACING [OVERWRITE_TARGET_SPACING ...]]
                                    [-overwrite_plans_name OVERWRITE_PLANS_NAME]
                                    [-c C [C ...]] [-np NP [NP ...]]
                                    [--verbose]

options:
  -h, --help            show this help message and exit
  -d D [D ...]          [REQUIRED] List of dataset IDs. Example: 2 4 5. This
                        will run fingerprint extraction, experiment planning
                        and preprocessing for these datasets. Can of course
                        al

In [12]:
# import os

# os.environ['nnUNet_raw'] = 'C:/Users/CUTY/Desktop/TO DELETE/AOCR2024/nnU_Base/nnUNet_raw_data'
# os.environ['nnUNet_preprocessed'] = 'C:/Users/CUTY/Desktop/TO DELETE/AOCR2024/nnU_Base/nnUNet_preprocessed'
# os.environ['nnUNet_results'] = 'C:/Users/CUTY/Desktop/TO DELETE/AOCR2024/nnU_Base/nnUNet_trained_models'

In [13]:
# import os
# print(os.path.abspath("./nnU_Base/nnUNet_raw_data/Task001_MYTASK"))
# print(os.listdir("./nnU_Base/nnUNet_raw_data"))
# print(os.listdir("./nnU_Base/nnUNet_raw_data/Task001_MYTASK"))

In [14]:
import os
os.environ['nnUNet_raw'] = r'C:\Users\CUTY\Desktop\AOR\nnU_Base\nnUNet_raw_data'
os.environ['nnUNet_preprocessed'] = r'C:\Users\CUTY\Desktop\AOR\nnU_Base\nnUNet_preprocessed'
os.environ['nnUNet_results'] = r'C:\Users\CUTY\Desktop\AOR\nnU_Base\nnUNet_trained_models'

# import sys
# print(sys.executable)

!nnUNetv2_extract_fingerprint -d 1 --verify_dataset_integrity

Dataset001_MYTASK
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> reader/writer

####################
verify_dataset_integrity Done. 
If you didn't see any error messages then your dataset is most likely OK!
####################



In [15]:
import os
print(os.getcwd())
!nnUNetv2_plan_experiment -d 1 -pl nnUNetPlannerResEncL

c:\Users\CUTY\Desktop\AOR
Attempting to find 3d_lowres config. 
Current spacing: [5.         0.70410156 0.70410156]. 
Current patch size: (np.int64(56), np.int64(320), np.int64(256)). 
Current median shape: [ 94.         497.08737864 497.08737864]
Attempting to find 3d_lowres config. 
Current spacing: [5.         0.72522461 0.72522461]. 
Current patch size: (np.int64(56), np.int64(320), np.int64(256)). 
Current median shape: [ 94.         482.60910548 482.60910548]
Attempting to find 3d_lowres config. 
Current spacing: [5.         0.74698135 0.74698135]. 
Current patch size: (np.int64(64), np.int64(256), np.int64(256)). 
Current median shape: [ 94.         468.55252959 468.55252959]
Attempting to find 3d_lowres config. 
Current spacing: [5.         0.76939079 0.76939079]. 
Current patch size: (np.int64(64), np.int64(256), np.int64(256)). 
Current median shape: [ 94.         454.90536853 454.90536853]
Attempting to find 3d_lowres config. 
Current spacing: [5.         0.79247251 0.792472

In [16]:
!nnUNetv2_preprocess -h

usage: nnUNetv2_preprocess [-h] [-d D [D ...]] [-plans_name PLANS_NAME]
                           [-c C [C ...]] [-np NP [NP ...]] [--verbose]

options:
  -h, --help            show this help message and exit
  -d D [D ...]          [REQUIRED] List of dataset IDs. Example: 2 4 5. This
                        will run fingerprint extraction, experiment planning
                        and preprocessing for these datasets. Can of course
                        also be just one dataset
  -plans_name PLANS_NAME
                        [OPTIONAL] You can use this to specify a custom plans
                        file that you may have generated
  -c C [C ...]          [OPTIONAL] Configurations for which the preprocessing
                        should be run. Default: 2d 3d_fullres 3d_lowres.
                        3d_cascade_fullres does not need to be specified
                        because it uses the data from 3d_fullres.
                        Configurations that do not exist for 

In [17]:
!nnUNetv2_preprocess -d 1 \
                     -plans_name nnUNetResEncUNetLPlans \
                     -c 3d_fullres

Preprocessing dataset Dataset001_MYTASK
Configuration: 3d_fullres...



100%|██████████| 300/300 [13:30<00:00,  2.70s/it]


## train

In [18]:
!nnUNetv2_train -h

usage: nnUNetv2_train [-h] [-tr TR] [-p P]
                      [-pretrained_weights PRETRAINED_WEIGHTS]
                      [-num_gpus NUM_GPUS] [--npz] [--c] [--val] [--val_best]
                      [--disable_checkpointing] [-device DEVICE]
                      dataset_name_or_id configuration fold

positional arguments:
  dataset_name_or_id    Dataset name or ID to train with
  configuration         Configuration that should be trained
  fold                  Fold of the 5-fold cross-validation. Should be an int
                        between 0 and 4.

options:
  -h, --help            show this help message and exit
  -tr TR                [OPTIONAL] Use this flag to specify a custom trainer.
                        Default: nnUNetTrainer
  -p P                  [OPTIONAL] Use this flag to specify a custom plans
                        identifier. Default: nnUNetPlans
  -pretrained_weights PRETRAINED_WEIGHTS
                        [OPTIONAL] path to nnU-Net checkpoint file 

In [27]:
import os
os.environ["NNUNET_MAX_EPOCHS"] = "10"

!nnUNetv2_train 1 3d_fullres 1 \
                -p nnUNetResEncUNetLPlans \
                --npz

^C


Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

2025-05-16 14:18:21.412077: do_dummy_2d_data_aug: True
2025-05-16 14:18:21.415077: Using splits from existing split file: C:\Users\CUTY\Desktop\AOR\nnU_Base\nnUNet_preprocessed\Dataset001_MYTASK\splits_final.json
2025-05-16 14:18:21.425078: The split file contains 5 splits.
2025-05-16 14:18:21.435081: Desired fold for training: 1
2025-05-16 14:18:21.442080: This split has 240 training and 60 validation cases.
using pin_memory on device 0
using pin_memory on device 0

This is the configuration used by this training:
Configuration name: 3d_fullres
 {'data_identifier': 'nn

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "C:\Users\CUTY\AppData\Local\Programs\Python\Python312\Lib\site-packages\batchgenerators\dataloading\nondet_multi_threaded_augmenter.py", line 53, in producer
    item = next(data_loader)
           ^^^^^^^^^^^^^^^^^
  File "C:\Users\CUTY\AppData\Local\Programs\Python\Python312\Lib\site-packages\batchgenerators\dataloading\data_loader.py", line 126, in __next__
    return self.generate_train_batch()
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\CUTY\AppData\Local\Programs\Python\Python312\Lib\site-packages\nnunetv2\training\dataloading\data_loader.py", line 170, in generate_train_batch
    data_all = np.zeros(self.data_shape, dtype=np.float32)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
numpy._core._exceptions._ArrayMemoryError: Unable to allocate 30.2 MiB for an array with shape (1

## find best configuration

In [ ]:
# !nnUNetv2_find_best_configuration -h

usage: nnUNetv2_find_best_configuration [-h] [-p P [P ...]] [-c C [C ...]]
                                        [-tr TR [TR ...]] [-np NP]
                                        [-f F [F ...]] [--disable_ensembling]
                                        [--no_overwrite]
                                        dataset_name_or_id

positional arguments:
  dataset_name_or_id    Dataset Name or id

options:
  -h, --help            show this help message and exit
  -p P [P ...]          List of plan identifiers. Default: nnUNetPlans
  -c C [C ...]          List of configurations. Default: ['2d', '3d_fullres',
                        '3d_lowres', '3d_cascade_fullres']
  -tr TR [TR ...]       List of trainers. Default: nnUNetTrainer
  -np NP                Number of processes to use for ensembling,
                        postprocessing etc
  -f F [F ...]          Folds to use. Default: 0 1 2 3 4
  --disable_ensembling  Set this flag to disable ensembling
  --no_overwrite        If set w

In [ ]:
# !nnUNetv2_find_best_configuration 1 \
#     -p nnUNetResEncUNetLPlans \
#     -c 3d_fullres \
#     -f 1
#     #-f 0 1 3

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\CUTY\AppData\Local\Programs\Python\Python312\Scripts\nnUNetv2_find_best_configuration.exe\__main__.py", line 7, in <module>
  File "C:\Users\CUTY\AppData\Local\Programs\Python\Python312\Lib\site-packages\nnunetv2\evaluation\find_best_configuration.py", line 296, in find_best_configuration_entry_point
    find_best_configuration(dataset_name, model_dict, allow_ensembling=not args.disable_ensembling,
  File "C:\Users\CUTY\AppData\Local\Programs\Python\Python312\Lib\site-packages\nnunetv2\evaluation\find_best_configuration.py", line 101, in find_best_configuration
    accumulate_cv_results(output_folder, merged_output_folder, folds, num_processes, overwrite)
  File "C:\Users\CUTY\AppData\Local\Programs\Python\Python312\Lib\site-packages\nnunetv2\evaluation\accumulate_cv_results.py", line 36, in accumulate_cv_results
    raise RuntimeE

## predict

In [ ]:
# !cp -r '/content/drive/MyDrive/poster/NNUNET/predict/input_partial' '/content'

'cp' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


In [35]:
!nnUNetv2_predict -i "C:\\Users\\CUTY\\Desktop\\AOR\\nnU_Base\\nnUNet_raw_data\\Dataset001_MYTASK\\imagesTs" \
                  -o "C:\\Users\\CUTY\\Desktop\\AOR\\nnU_Base\\nnUnet_predictions" \
                  -d 1 -c 3d_fullres -f 1 -p nnUNetResEncUNetLPlans --disable_tta



#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

There are 20 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 20 cases that I would like to predict

Predicting AA:
perform_everything_on_device: True
sending off prediction to background worker for resampling and export
done with AA

Predicting AB:
perform_everything_on_device: True
sending off prediction to background worker for resampling and export
done with AB

Predicting AC:
perform_everything_on_device: True
sending off prediction to background worker for resampling and export
done with AC

Predicting AD:
perform_everything


100%|██████████| 18/18 [00:10<00:00,  1.79it/s]

100%|██████████| 27/27 [00:09<00:00,  2.98it/s]

100%|██████████| 36/36 [00:12<00:00,  2.95it/s]

100%|██████████| 36/36 [00:12<00:00,  2.95it/s]

100%|██████████| 27/27 [00:08<00:00,  3.07it/s]

100%|██████████| 36/36 [00:12<00:00,  2.92it/s]

100%|██████████| 36/36 [00:12<00:00,  2.95it/s]

100%|██████████| 18/18 [00:05<00:00,  3.39it/s]

100%|██████████| 36/36 [00:12<00:00,  2.91it/s]

100%|██████████| 36/36 [00:12<00:00,  2.90it/s]

100%|██████████| 36/36 [00:12<00:00,  2.86it/s]

100%|██████████| 36/36 [00:12<00:00,  2.92it/s]

100%|██████████| 27/27 [00:08<00:00,  3.03it/s]

100%|██████████| 27/27 [00:08<00:00,  3.05it/s]

100%|██████████| 27/27 [00:09<00:00,  3.00it/s]

100%|██████████| 36/36 [00:12<00:00,  2.81it/s]

100%|██████████| 36/36 [00:12<00:00,  2.80it/s]

100%|██████████| 27/27 [00:09<00:00,  2.96it/s]

100%|██████████| 27/27 [00:09<00:00,  2.97it/s]

100%|██████████| 27/27 [00:09<00:00,  2.96it/s]


In [ ]:
# !nnUNetv2_predict -d Dataset003_aocr2024_partial \
#     -i "/content/input_partial" \
#     -o "/content/drive/MyDrive/poster/NNUNET/predict/Dataset003_aocr2024_partial/nnUNetTrainer__nnUNetResEncUNetLPlans__3d_fullres/fold_0_1_3/output" \
#     -f  0 1 3 \
#     -tr nnUNetTrainer \
#     -c 3d_fullres \
#     -p nnUNetResEncUNetLPlans


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################



Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\CUTY\AppData\Local\Programs\Python\Python312\Scripts\nnUNetv2_predict.exe\__main__.py", line 7, in <module>
  File "C:\Users\CUTY\AppData\Local\Programs\Python\Python312\Lib\site-packages\nnunetv2\inference\predict_from_raw_data.py", line 979, in predict_entry_point
    predictor.initialize_from_trained_model_folder(
  File "C:\Users\CUTY\AppData\Local\Programs\Python\Python312\Lib\site-packages\nnunetv2\inference\predict_from_raw_data.py", line 76, in initialize_from_trained_model_folder
    dataset_json = load_json(join(model_training_output_dir, 'dataset.json'))
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\CUTY\AppData\Local\Programs\Python\Python312\Lib\site-packages\batchgenerators\utilities\file_and_folder_operations.py", line 103, in load_json
    with open(file, 'r') as f:
 

## postprocess

In [37]:
import os
import nibabel as nib
import numpy as np
import pandas as pd

# 設定測試集預測結果資料夾
pred_dir = r"C:\Users\CUTY\Desktop\AOR\nnU_Base\nnUnet_predictions"

# 儲存 slice-level labels 的 CSV 路徑
output_csv_path = r"C:\Users\CUTY\Desktop\AOR\scan_slice_level_predictions.csv"

# 建立用來儲存 CSV 資料的列表
records = []

# 遍歷預測資料夾，處理每個 .nii.gz 檔案
for filename in os.listdir(pred_dir):
    if filename.endswith('.nii.gz'):
        pred_filepath = os.path.join(pred_dir, filename)
        pred_img = nib.load(pred_filepath)
        pred_data = pred_img.get_fdata()

        # 提取 Scan name (假設檔案命名如 Sample_0, Sample_1, 等等)
        scan_name = filename.split('_')[0]

        # 根據 slice-level 預測推算 scan-level 標籤
        # 只要任一 slice 預測為發炎（label=1），就視為整個 scan 為發炎
        slices_with_inflammation = (pred_data == 1).sum(axis=(0, 1)) > 0
        scan_pred = 1 if np.any(slices_with_inflammation) else 0

        # 記錄 scan-level 的資料
        records.append({
            "id": scan_name,  # scan-level id（例如 Sample）
            "label": scan_pred  # scan-level 預測結果
        })

        # 逐切片處理，檢查每個 slice 是否預測為發炎區
        for slice_idx in range(pred_data.shape[2]):
            # 檢查這個 slice 是否有發炎（即 label=1）
            slice_data = pred_data[:, :, slice_idx]
            slice_label = 1 if np.any(slice_data == 1) else 0  # 只要有任何像素為 1，就判定為發炎區

            # 記錄 slice-level 資料
            records.append({
                "id": f"{scan_name}_{slice_idx}",  # 生成 slice 的 id（例如 Sample_0）
                "label": slice_label  # 這個 slice 的標籤
            })

# 儲存為 CSV
df = pd.DataFrame(records)
df.to_csv(output_csv_path, index=False)

print(f"成功將預測結果儲存為 scan-slice-level labels CSV：{output_csv_path}")


成功將預測結果儲存為 scan-slice-level labels CSV：C:\Users\CUTY\Desktop\AOR\scan_slice_level_predictions.csv


In [ ]:
# import pandas as pd
# from sklearn.metrics import precision_recall_fscore_support

# # 讀取 ground truth 的 slice-level labels
# gt_path = r"C:\Users\CUTY\Desktop\AOR\test_team1_ground_truth.csv"
# gt_df = pd.read_csv(gt_path)

# # 讀取預測的 slice-level labels
# pred_path = r"C:\Users\CUTY\Desktop\AOR\scan_slice_level_predictions.csv"
# pred_df = pd.read_csv(pred_path)

# # 處理預測資料的 id 格式，將 ".nii.gz" 去掉以便與 ground truth 的 id 格式一致
# # 但保留 slice 索引（例如 AA_0, AA_1）
# pred_df['id'] = pred_df['id'].apply(lambda x: x.replace('.nii.gz', ''))  # 保留主檔名，不去掉 slice 索引

# # 確保預測和 ground truth 的 id 在合併時匹配
# # Ground truth 的 id 會有 AA, AA_0, AA_1, AA_2, ... 所以需要處理這個
# # 根據相同的主檔名來將預測結果對應到每個 slice
# gt_df['id'] = gt_df['id'].apply(lambda x: x.split('.')[0])  # 同樣處理 ground truth，去掉 .nii.gz

# # 檢查資料是否正確載入
# print("Ground Truth Data:")
# print(gt_df.head())  # 顯示 ground truth 的前幾行
# print("Prediction Data:")
# print(pred_df.head())  # 顯示預測結果的前幾行

# # 1. 計算 slice-level F1
# # 比較預測結果與 ground truth
# y_true = gt_df['label'].values
# y_pred = pred_df['label'].values

# # 計算 F1 分數
# precision_s, recall_s, f1_s, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')

# print(f"Slice-level Precision: {precision_s:.4f}")
# print(f"Slice-level Recall: {recall_s:.4f}")
# print(f"Slice-level F1 Score: {f1_s:.4f}")

# # 2. 計算 scan-level F1
# # 根據 slice-level 預測來推算 scan-level 預測
# # 如果該 scan 中任一 slice 預測為發炎，則整個 scan 視為發炎
# scan_gt = {}
# scan_pred = {}

# # 取得 scan-level ground truth
# current_scan = None
# for idx, row in gt_df.iterrows():
#     if '_' not in row['id']:  # scan-level行
#         current_scan = row['id']
#         scan_gt[current_scan] = row['label']

# # 取得 slice-level 預測
# for idx, row in pred_df.iterrows():
#     scan_name = row['id'].split('_')[0]  # Scan 名稱是 slice 名稱的前綴（例如 Sample_0 -> Sample）
#     slice_label = row['label']

#     # 根據所有 slice 預測來決定 scan-level 預測
#     if scan_name not in scan_pred:
#         scan_pred[scan_name] = 0  # 初始化為正常（0）

#     if slice_label == 1:
#         scan_pred[scan_name] = 1  # 只要一個 slice 被預測為發炎，scan 就是發炎

# # 3. 計算 scan-level F1
# scan_gt_labels = list(scan_gt.values())
# scan_pred_labels = [scan_pred.get(scan, 0) for scan in scan_gt.keys()]

# # 如果 scan_pred_labels 和 scan_gt_labels 都有有效的預測，計算 F1
# if len(scan_pred_labels) > 0 and len(scan_gt_labels) > 0:
#     precision_sc, recall_sc, f1_sc, _ = precision_recall_fscore_support(scan_gt_labels, scan_pred_labels, average='binary')
# else:
#     precision_sc, recall_sc, f1_sc = 0, 0, 0

# print(f"\nScan-level Precision: {precision_sc:.4f}")
# print(f"Scan-level Recall: {recall_sc:.4f}")
# print(f"Scan-level F1 Score: {f1_sc:.4f}")

# # 4. 合併 ground truth 和預測結果
# # 合併兩個資料框
# merged_df = pd.merge(gt_df, pred_df, on='id', suffixes=('_gt', '_pred'))

# # 檢查合併後的結果
# print("Merged DataFrame:")
# print(merged_df.head())  # 顯示合併後的前幾行

# # 計算每個 slice 的 TP, FP, FN 和 F1
# merged_df['TP'] = ((merged_df['label_gt'] == 1) & (merged_df['label_pred'] == 1)).astype(int)
# merged_df['FP'] = ((merged_df['label_gt'] == 0) & (merged_df['label_pred'] == 1)).astype(int)
# merged_df['FN'] = ((merged_df['label_gt'] == 1) & (merged_df['label_pred'] == 0)).astype(int)

# # 計算每個 slice 的 precision, recall, f1
# merged_df['Precision'] = merged_df['TP'] / (merged_df['TP'] + merged_df['FP']) if (merged_df['TP'] + merged_df['FP']).sum() > 0 else 0
# merged_df['Recall'] = merged_df['TP'] / (merged_df['TP'] + merged_df['FN']) if (merged_df['TP'] + merged_df['FN']).sum() > 0 else 0
# merged_df['F1'] = 2 * (merged_df['Precision'] * merged_df['Recall']) / (merged_df['Precision'] + merged_df['Recall']) if (merged_df['Precision'] + merged_df['Recall']).sum() > 0 else 0

# # 保存合併結果到 CSV
# output_csv_path = r"C:\Users\CUTY\Desktop\AOR\final_results_with_f1.csv"
# merged_df.to_csv(output_csv_path, index=False)

# print(f"\nMerged and saved results to: {output_csv_path}")

# # 平均 F1 分數
# avg_f1_slice = merged_df['F1'].mean()
# print(f"\nAverage F1 Score for Slice-level: {avg_f1_slice:.4f}")


Ground Truth Data:
     id  label
0    AA      1
1  AA_0      0
2  AA_1      0
3  AA_2      0
4  AA_3      0
Prediction Data:
     id  label
0    AA      1
1  AA_0      0
2  AA_1      0
3  AA_2      1
4  AA_3      1
Slice-level Precision: 0.0682
Slice-level Recall: 0.7255
Slice-level F1 Score: 0.1247

Scan-level Precision: 0.5000
Scan-level Recall: 1.0000
Scan-level F1 Score: 0.6667
Merged DataFrame:
     id  label_gt  label_pred
0    AA         1           1
1  AA_0         0           0
2  AA_1         0           0
3  AA_2         0           1
4  AA_3         0           1

Merged and saved results to: C:\Users\CUTY\Desktop\AOR\final_results_with_f1.csv

Average F1 Score for Slice-level: 1.0000


In [44]:
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support

# 讀取 ground truth 的 slice-level labels
gt_path = r"C:\Users\CUTY\Desktop\AOR\test_team1_ground_truth.csv"
gt_df = pd.read_csv(gt_path)

# 讀取預測的 slice-level labels
pred_path = r"C:\Users\CUTY\Desktop\AOR\scan_slice_level_predictions.csv"
pred_df = pd.read_csv(pred_path)

# 處理預測資料的 id 格式，將 ".nii.gz" 去掉以便與 ground truth 的 id 格式一致
# 但保留 slice 索引（例如 AA_0, AA_1）
pred_df['id'] = pred_df['id'].apply(lambda x: x.replace('.nii.gz', ''))  # 保留主檔名，不去掉 slice 索引

# 確保預測和 ground truth 的 id 在合併時匹配
# Ground truth 的 id 會有 AA, AA_0, AA_1, AA_2, ... 所以需要處理這個
# 根據相同的主檔名來將預測結果對應到每個 slice
gt_df['id'] = gt_df['id'].apply(lambda x: x.split('.')[0])  # 同樣處理 ground truth，去掉 .nii.gz

# 1. 計算 slice-level F1
# 比較預測結果與 ground truth
y_true = gt_df['label'].values
y_pred = pred_df['label'].values

# 計算 F1 分數
precision_s, recall_s, f1_s, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')

print(f"Slice-level F1 Score: {f1_s:.4f}")

# 2. 計算 scan-level F1
# 根據 slice-level 預測來推算 scan-level 預測
# 如果該 scan 中任一 slice 預測為發炎，則整個 scan 視為發炎
scan_gt = {}
scan_pred = {}

# 取得 scan-level ground truth
current_scan = None
for idx, row in gt_df.iterrows():
    if '_' not in row['id']:  # scan-level行
        current_scan = row['id']
        scan_gt[current_scan] = row['label']

# 取得 slice-level 預測
for idx, row in pred_df.iterrows():
    scan_name = row['id'].split('_')[0]  # Scan 名稱是 slice 名稱的前綴（例如 Sample_0 -> Sample）
    slice_label = row['label']

    # 根據所有 slice 預測來決定 scan-level 預測
    if scan_name not in scan_pred:
        scan_pred[scan_name] = 0  # 初始化為正常（0）

    if slice_label == 1:
        scan_pred[scan_name] = 1  # 只要一個 slice 被預測為發炎，scan 就是發炎

# 3. 計算 scan-level F1
scan_gt_labels = list(scan_gt.values())
scan_pred_labels = [scan_pred.get(scan, 0) for scan in scan_gt.keys()]

# 計算 Scan-level F1
precision_sc, recall_sc, f1_sc, _ = precision_recall_fscore_support(scan_gt_labels, scan_pred_labels, average='binary')

print(f"Scan-level F1 Score: {f1_sc:.4f}")

# 4. 合併 ground truth 和預測結果
# 合併兩個資料框
merged_df = pd.merge(gt_df, pred_df, on='id', suffixes=('_gt', '_pred'))

# 計算每個 slice 的 F1
merged_df['TP'] = ((merged_df['label_gt'] == 1) & (merged_df['label_pred'] == 1)).astype(int)
merged_df['FP'] = ((merged_df['label_gt'] == 0) & (merged_df['label_pred'] == 1)).astype(int)
merged_df['FN'] = ((merged_df['label_gt'] == 1) & (merged_df['label_pred'] == 0)).astype(int)

# 計算每個 slice 的 f1
merged_df['F1'] = 2 * (merged_df['TP'] * merged_df['TP']) / (merged_df['TP'] + merged_df['TP'])

# 保存合併結果到 CSV
output_csv_path = r"C:\Users\CUTY\Desktop\AOR\final_results_with_f1.csv"
merged_df.to_csv(output_csv_path, index=False)

print(f"Finished saving F1 Score to: {output_csv_path}")


Slice-level F1 Score: 0.1247
Scan-level F1 Score: 0.6667
Finished saving F1 Score to: C:\Users\CUTY\Desktop\AOR\final_results_with_f1.csv


In [ ]:
# !nnUNetv2_apply_postprocessing \
#     -i '/content/drive/MyDrive/poster/NNUNET/predict/Dataset003_aocr2024_partial/nnUNetTrainer__nnUNetResEncUNetLPlans__3d_fullres/fold_0_1_3/output' \
#     -o '/content/drive/MyDrive/poster/NNUNET/predict/Dataset003_aocr2024_partial/nnUNetTrainer__nnUNetResEncUNetLPlans__3d_fullres/fold_0_1_3/output_pp' \
#     -pp_pkl_file /content/drive/MyDrive/poster/NNUNET/nnUNet_results/Dataset003_aocr2024_partial/nnUNetTrainer__nnUNetResEncUNetLPlans__3d_fullres/crossval_results_folds_0_1_3/postprocessing.pkl \
#     -np 8 \
#     -plans_json /content/drive/MyDrive/poster/NNUNET/nnUNet_results/Dataset003_aocr2024_partial/nnUNetTrainer__nnUNetResEncUNetLPlans__3d_fullres/crossval_results_folds_0_1_3/plans.json

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\CUTY\AppData\Local\Programs\Python\Python312\Scripts\nnUNetv2_apply_postprocessing.exe\__main__.py", line 7, in <module>
  File "C:\Users\CUTY\AppData\Local\Programs\Python\Python312\Lib\site-packages\nnunetv2\postprocessing\remove_connected_components.py", line 332, in entry_point_apply_postprocessing
    pp_fns, pp_fn_kwargs = load_pickle(args.pp_pkl_file)
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\CUTY\AppData\Local\Programs\Python\Python312\Lib\site-packages\batchgenerators\utilities\file_and_folder_operations.py", line 92, in load_pickle
    with open(file, mode) as f:
         ^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/poster/NNUNET/nnUNet_results/Dataset003_aocr2024_partial/nnUNetTrainer__nnUNetResEncUNetLPlans__3d_fullres/crossval_results_f